# About the BETYdb service in the PEcAn platform

According to the [BETYdb documentation](https://pecan.gitbook.io/betydbdoc-dataentry), 
BETYdb is used to manage and distribute agricultural and ecological data. 
The web interface provides a sequence of pages that walk through the process of entering meta-data, 
and then the option of entering trait and yield data through a similar web form or via upload of a text (csv) file. 
For entering large tables of data there is Bulk Upload Wizard. 
This is useful when entering more than a few dozen trait or yield data from a single source.

There are typically two categories of data that are entered:

- Results from previously published research, typically statistical summaries.
- Primary data, observations at the level of an individual replicate. 

BETYdb supports both of these, because it was designed to support new research that quantitatively builds on previous research, 
allowing researchers to develop, test, and evaluate new hypotheses based on what is already known. 


# Install BetyDB with helm

We will create a `pecan-betydb` Secret from an existing `pecan-postgresql` database Secret that BETYdb will use to connect to PostgreSQL. 

In [ ]:
%%bash
oc delete secret/pecan-betydb
oc create secret generic pecan-betydb \
  --from-literal=postgresqlPassword=$(oc get secret pecan-postgresql -o jsonpath={.data.postgres-password} | base64 -d) \
  --from-literal=betyPassword=$(oc get secret pecan-postgresql -o jsonpath={.data.postgres-password} | base64 -d) \
  --from-literal=secretKey="thisisnotasecret"
echo DONE

Run the commands below to install BetyDB on top of PostgreSQL for ecological forecasting data. 

In [ ]:
%%bash
git clone https://github.com/computate-org/bety-helm.git ~/.local/src/bety-helm/
helm repo add bitnami https://raw.githubusercontent.com/bitnami/charts/archive-full-index/bitnami
(cd ~/.local/src/bety-helm/ && helm dependency build)
helm upgrade --install betydb ~/.local/src/bety-helm/  \
  --set image.repository=quay.io/computateorg/bety \
  --set image.tag=latest \
  --set image.pullPolicy=Always \
  --set postgres.enabled=false \
  --set postgresql.enabled=false \
  --set postgresql.fullnameOverride=pecan-postgresql \
  --set postgresql.serviceAccount.enabled=true \
  --set postgresql.postgresqlHost=pecan-postgresql \
  --set serviceAccount.enabled=true \
  --set serviceAccount.name=pecan-betydb \
  --set externalSecret=true \
  --set nameOverride=pecan \
  --set fullnameOverride=pecan-betydb
echo DONE

### View BETYdb pod details
After deploying the Helm Chart, it will take a minute before the PostgreSQL pod is up and running. Run the command below until the PostgreSQL pod health checks are `READY 1/1` and `STATUS Running`. 
You may also see a `pecan-betydb-load-db` Job that also completed.

In [ ]:
%%bash
oc get pod -l app.kubernetes.io/instance=betydb
echo DONE

### View BETYdb pod logs
If your BETYdb pod does not reach the STATUS Running, you can run the command below to view the pod logs of BETYdb and check for other errors that may have occured. 

In [ ]:
%%bash
oc logs -l app.kubernetes.io/instance=betydb
echo DONE

## Check the version of BETYdb deployed

BetyDB is an open source project in the PEcAn platform, and we are using a custom image for BETYdb that is being tested for compatibility with other components. 
You can check on the image version of RabbitMQ by checking the `image` of the `Deployment/pecan-rabbitmq` below. 

In [ ]:
%%bash
oc get deployment/pecan-betydb -o yaml | grep image:
echo DONE

The image you discovered above can also be found in quay.io here: 

https://quay.io/repository/computateorg/bety

## More information about deploying BETYdb

To learn more about deploying BETYdb using the Bitnami Helm Chart see: 

https://github.com/PecanProject/bety-helm

## Next...
I hope that answers your questions about BETYdb in the PEcAn Platform. 
- If you have additional questions or issues, please [create an issue for the course here](https://github.com/computate-org/pecan-unconstrained-forecast-course/issues). 
- Otherwise, please continue to the next notebook [06-install-pecan.ipynb](06-install-pecan.ipynb). 